# Python 資料分析

> 以 Pandas 處理表格式資料

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
import datetime
import urllib
import requests
import numpy as np

## 關於 Pandas

## 什麼是 Pandas？

> Pandas 是 Python 處理表格式資料的第三方模組（套件），它創造了稱為索引（Index）、序列（Series）與資料框（DataFrame）的資料結構，透過這些資料結構，可以讓 Python 在面對文字檔案、Excel 試算表與關聯式資料庫時能夠使用更直覺的觀念操作，其中資料框啟發自 R 語言。

來源：<https://github.com/pandas-dev/pandas>

## 為什麼資料分析採用 Pandas？

在 Pandas 誕生之前（2008 年）Python 並沒有一個合適的資料結構來處理資料分析常見的「表格式資料（Tabular datasets）」，Python 使用者必須在工作流程中切換到像是 R 語言或者 Matlab。

## 以 `import` 指令載入 Pandas

`pandas` 的官方縮寫為 `pd`。

In [2]:
import pandas as pd

## 如果環境中沒有安裝 Pandas，載入時會遭遇 `ModuleNotFoundError`

```
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'pandas'
```

## 如果遭遇 `ModuleNotFoundError` 可以在終端機使用 `pip install` 指令安裝

```bash
pip install pandas
```

## 可以透過兩個屬性檢查版本號與安裝路徑

- `__version__` 屬性檢查版本號。
- `__file__` 屬性檢查安裝路徑。

In [3]:
print(pd.__version__)
print(pd.__file__)

1.2.4
/Users/kuoyaojen/opt/miniconda3/envs/pyds/lib/python3.8/site-packages/pandas/__init__.py


## （沒什麼用的）冷知識：Pandas 跟熊貓的關係是？

![](https://media.giphy.com/media/46Zj6ze2Z2t4k/giphy.gif)

來源：<https://giphy.com>

## 與熊貓完全沒有關係，Pandas 是由三個起初創造的資料結構類別名稱組合而成

- **Pan**el（自從版本 0.20.0 之後停用）
- **Da**taFrame
- **S**eries

## Pandas 起步走

## 想要精通 Pandas，必須掌握 `Index`、`ndarray`、`Series` 與 `DataFrame` 四個類別之間的關係

- `Series` 由 `Index` 與 `ndarray` 組合而成。
- `DataFrame` 由數個共享同一個 `Index` 的 `Series` 組合而成。

## Pandas 的 `Index` 類別

使用 `Index` 函數創造 `Index` 類別的實例。

In [4]:
prime_indices = pd.Index([2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
print(type(prime_indices))

<class 'pandas.core.indexes.numeric.Int64Index'>


## `Index` 類別像是 Python 內建 `tuple` 與 `set` 兩種類別的結合

- 具有 `tuple` 無法更動的特性。
- 具有 `set` 集合運算的特性。 

In [5]:
# Index has the characteristics of a tuple
prime_indices = pd.Index([2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
try:
    prime_indices[-1] = 31
except:
    print("Immutable!")

Immutable!


In [6]:
# Index has the characteristics of a set
odd_indices = pd.Index(range(1, 30, 2))
print(prime_indices.intersection(odd_indices))         # prime_indices & odd_indices
print(prime_indices.union(odd_indices))                # prime_indices | odd_indices
print(prime_indices.symmetric_difference(odd_indices)) # prime_indices ^ odd_indices
print(prime_indices.difference(odd_indices))
print(odd_indices.difference(prime_indices))

Int64Index([3, 5, 7, 11, 13, 17, 19, 23, 29], dtype='int64')
Int64Index([1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29], dtype='int64')
Int64Index([1, 2, 9, 15, 21, 25, 27], dtype='int64')
Int64Index([2], dtype='int64')
Int64Index([1, 9, 15, 21, 25, 27], dtype='int64')


## Pandas 的 `Series` 類別

使用 `Series` 函數創造 `Series` 類別的實例。

In [7]:
prime_series = pd.Series([2, 3, 5, 7, 11, 13, 17, 19, 23, 29])
print(type(prime_series))

<class 'pandas.core.series.Series'>


## `Series` 由 `Index` 與 `ndarray` 組合而成

In [8]:
print(type(prime_series.index))
print(type(prime_series.values))

<class 'pandas.core.indexes.range.RangeIndex'>
<class 'numpy.ndarray'>


## Pandas 的 `DataFrame` 類別

使用 `DataFrame` 函數創造 `DataFrame` 類別的實例。

In [9]:
movie_df = pd.DataFrame()
movie_df["title"] = ["The Shawshank Redemption", "The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
movie_df["imdb_rating"] = [9.3, 9.0, 8.9, 8.8, 8.7]
print(type(movie_df))

<class 'pandas.core.frame.DataFrame'>


## `DataFrame` 由數個 `Series` 共享同一個 `Index` 組成

In [10]:
print(type(movie_df.index))
print(type(movie_df["title"]))
print(type(movie_df["imdb_rating"]))

<class 'pandas.core.indexes.range.RangeIndex'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


## Pandas 提供函數、類別屬性與方法處理表格式資料的

- 載入（Import）
- 清理（Tidy）
- 轉型（Transform）

## `DataFrame` 的基礎屬性和方法

In [11]:
def get_latest_daily_report():
    """
    This function returns the latest global daily report from https://github.com/CSSEGISandData
    /COVID-19 and its file date.
    """
    latest_date = datetime.date.today()
    day_delta = datetime.timedelta(days=1)
    fmt = '%m-%d-%Y'
    while True:
        try:
            latest_date_fmt = latest_date.strftime(fmt)
            csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master\
/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(latest_date_fmt)
            daily_report = pd.read_csv(csv_url)
            print("載入了 {} 的每日報告。".format(latest_date_fmt))
            break
        except urllib.error.HTTPError:
            latest_date_fmt = latest_date.strftime(fmt)
            print("尚未有 {} 的每日報告。".format(latest_date_fmt))
            latest_date -= day_delta
    return daily_report

In [12]:
daily_report = get_latest_daily_report()

尚未有 09-24-2021 的每日報告。
載入了 09-23-2021 的每日報告。


## `DataFrame` 的基礎屬性

- `shape`
- `dtypes`
- `index`
- `columns`

In [13]:
print(daily_report.shape)
print(daily_report.dtypes)
print(daily_report.index)
print(daily_report.columns)

(4004, 14)
FIPS                   float64
Admin2                  object
Province_State          object
Country_Region          object
Last_Update             object
Lat                    float64
Long_                  float64
Confirmed                int64
Deaths                   int64
Recovered              float64
Active                 float64
Combined_Key            object
Incident_Rate          float64
Case_Fatality_Ratio    float64
dtype: object
RangeIndex(start=0, stop=4004, step=1)
Index(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incident_Rate', 'Case_Fatality_Ratio'],
      dtype='object')


## `DataFrame` 的基礎方法

- `head(n)`
- `tail(n)`
- `describe`
- `info`
- `set_index`
- `reset_index`

## `head(n)` 方法回傳欄位名稱與前 n 列觀測值

In [14]:
daily_report.head() # n is default to 5

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-09-24 04:21:44,33.93911,67.709953,154800,7199,NaN,NaN,Afghanistan,397.653730,4.650517
1,NaN,NaN,NaN,Albania,2021-09-24 04:21:44,41.15330,20.168300,165864,2609,NaN,NaN,Albania,5763.569393,1.572975
2,NaN,NaN,NaN,Algeria,2021-09-24 04:21:44,28.03390,1.659600,202283,5748,NaN,NaN,Algeria,461.295755,2.841564
3,NaN,NaN,NaN,Andorra,2021-09-24 04:21:44,42.50630,1.521800,15156,130,NaN,NaN,Andorra,19615.608620,0.857746
4,NaN,NaN,NaN,Angola,2021-09-24 04:21:44,-11.20270,17.873900,53840,1460,NaN,NaN,Angola,163.815374,2.711738


## `tail(n)` 方法回傳欄位名稱與後 n 列觀測值

In [15]:
daily_report.tail(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
4001,NaN,NaN,NaN,Yemen,2021-09-24 04:21:44,15.552727,48.516388,8830,1664,NaN,NaN,Yemen,29.605074,18.844847
4002,NaN,NaN,NaN,Zambia,2021-09-24 04:21:44,-13.133897,27.849332,208715,3641,NaN,NaN,Zambia,1135.310594,1.744484
4003,NaN,NaN,NaN,Zimbabwe,2021-09-24 04:21:44,-19.015438,29.154857,129134,4600,NaN,NaN,Zimbabwe,868.832902,3.562191


## `describe` 方法回傳數值欄位的描述性統計

In [16]:
daily_report.describe()

,FIPS,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
count,3267.000000,3914.000000,3914.000000,4.004000e+03,4004.000000,0.0,0.0,3914.000000,3964.000000
mean,32415.844200,35.831394,-71.300276,5.759280e+04,1181.031968,NaN,NaN,11798.842391,2.266735
std,18050.270924,13.196117,54.870853,3.216703e+05,6802.620838,NaN,NaN,5018.455527,21.044156
min,66.000000,-52.368000,-178.116500,0.000000e+00,0.000000,NaN,NaN,0.000000,0.000000
25%,19050.000000,33.212580,-96.580090,1.564500e+03,24.000000,NaN,NaN,8979.083298,1.115511
50%,30069.000000,37.910342,-86.752466,4.432500e+03,70.000000,NaN,NaN,12510.243710,1.589461
75%,47042.000000,42.181730,-77.393369,1.820450e+04,257.000000,NaN,NaN,15179.469425,2.227752
max,99999.000000,71.706900,178.065000,6.960267e+06,148688.000000,NaN,NaN,49060.236545,1286.772487


## `info` 方法回傳資料框的詳細資訊

In [17]:
daily_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4004 entries, 0 to 4003
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FIPS                 3267 non-null   float64
 1   Admin2               3272 non-null   object 
 2   Province_State       3831 non-null   object 
 3   Country_Region       4004 non-null   object 
 4   Last_Update          4004 non-null   object 
 5   Lat                  3914 non-null   float64
 6   Long_                3914 non-null   float64
 7   Confirmed            4004 non-null   int64  
 8   Deaths               4004 non-null   int64  
 9   Recovered            0 non-null      float64
 10  Active               0 non-null      float64
 11  Combined_Key         4004 non-null   object 
 12  Incident_Rate        3914 non-null   float64
 13  Case_Fatality_Ratio  3964 non-null   float64
dtypes: float64(7), int64(2), object(5)
memory usage: 438.1+ KB


## `set_index` 方法可以指定欄位取代目前的 `Index`

In [18]:
daily_report.set_index('Combined_Key')

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
Combined_Key,,,,,,,,,,,,,
Afghanistan,NaN,NaN,NaN,Afghanistan,2021-09-24 04:21:44,33.939110,67.709953,154800,7199,NaN,NaN,397.653730,4.650517
Albania,NaN,NaN,NaN,Albania,2021-09-24 04:21:44,41.153300,20.168300,165864,2609,NaN,NaN,5763.569393,1.572975
Algeria,NaN,NaN,NaN,Algeria,2021-09-24 04:21:44,28.033900,1.659600,202283,5748,NaN,NaN,461.295755,2.841564
Andorra,NaN,NaN,NaN,Andorra,2021-09-24 04:21:44,42.506300,1.521800,15156,130,NaN,NaN,19615.608620,0.857746
Angola,NaN,NaN,NaN,Angola,2021-09-24 04:21:44,-11.202700,17.873900,53840,1460,NaN,NaN,163.815374,2.711738
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,NaN,NaN,NaN,Vietnam,2021-09-24 04:21:44,14.058324,108.277199,728435,18017,NaN,NaN,748.351761,2.473385
West Bank and Gaza,NaN,NaN,NaN,West Bank and Gaza,2021-09-24 04:21:44,31.952200,35.233200,392452,3980,NaN,NaN,7693.001316,1.014137
Yemen,NaN,NaN,NaN,Yemen,2021-09-24 04:21:44,15.552727,48.516388,8830,1664,NaN,NaN,29.605074,18.844847


## `reset_index` 方法可以用預設的 `RangeIndex` 重設目前的 `Index`

In [19]:
daily_report.set_index('Combined_Key').reset_index()

,Combined_Key,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
0,Afghanistan,NaN,NaN,NaN,Afghanistan,2021-09-24 04:21:44,33.939110,67.709953,154800,7199,NaN,NaN,397.653730,4.650517
1,Albania,NaN,NaN,NaN,Albania,2021-09-24 04:21:44,41.153300,20.168300,165864,2609,NaN,NaN,5763.569393,1.572975
2,Algeria,NaN,NaN,NaN,Algeria,2021-09-24 04:21:44,28.033900,1.659600,202283,5748,NaN,NaN,461.295755,2.841564
3,Andorra,NaN,NaN,NaN,Andorra,2021-09-24 04:21:44,42.506300,1.521800,15156,130,NaN,NaN,19615.608620,0.857746
4,Angola,NaN,NaN,NaN,Angola,2021-09-24 04:21:44,-11.202700,17.873900,53840,1460,NaN,NaN,163.815374,2.711738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,Vietnam,NaN,NaN,NaN,Vietnam,2021-09-24 04:21:44,14.058324,108.277199,728435,18017,NaN,NaN,748.351761,2.473385
4000,West Bank and Gaza,NaN,NaN,NaN,West Bank and Gaza,2021-09-24 04:21:44,31.952200,35.233200,392452,3980,NaN,NaN,7693.001316,1.014137
4001,Yemen,NaN,NaN,NaN,Yemen,2021-09-24 04:21:44,15.552727,48.516388,8830,1664,NaN,NaN,29.605074,18.844847
4002,Zambia,NaN,NaN,NaN,Zambia,2021-09-24 04:21:44,-13.133897,27.849332,208715,3641,NaN,NaN,1135.310594,1.744484


## 基礎的資料框操作

## Pandas 使用更直觀的概念操作資料

- 如何定義「更直觀」？
    - 像操作試算表一樣（Spreadsheet-like）。
    - 像使用資料庫一樣（SQL-like）。

## 基礎的資料框操作語法

- 選擇。
- 篩選。
- 選擇與篩選。

## 基礎的資料框操作語法（續）

- 索引。
- 排序。
- 衍生計算欄位。
- 聚合。
- 分組聚合。

## 以 `['欄位名稱']` 語法選擇出資料框的欄位成為外型 `(m,)` 的 `Series`

In [20]:
print(daily_report['Country_Region'])
print(type(daily_report['Country_Region']))

0              Afghanistan
1                  Albania
2                  Algeria
3                  Andorra
4                   Angola
               ...        
3999               Vietnam
4000    West Bank and Gaza
4001                 Yemen
4002                Zambia
4003              Zimbabwe
Name: Country_Region, Length: 4004, dtype: object
<class 'pandas.core.series.Series'>


## 以 `[['欄位名稱']]` 語法選擇出資料框的欄位成為外型 `(m, 1)` 的 `DataFrame`

In [21]:
print(type(daily_report[['Country_Region']]))
daily_report[['Country_Region']]

<class 'pandas.core.frame.DataFrame'>


,Country_Region
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
3999,Vietnam
4000,West Bank and Gaza
4001,Yemen
4002,Zambia


## 以 `[['欄位名稱', '欄位名稱', ...]]` 語法選擇出資料框多個欄位成為外型 `(m, n)` 的 `DataFrame`

In [22]:
cols = ['Country_Region', 'Lat', 'Long_']
daily_report[cols]

,Country_Region,Lat,Long_
0,Afghanistan,33.939110,67.709953
1,Albania,41.153300,20.168300
2,Algeria,28.033900,1.659600
3,Andorra,42.506300,1.521800
4,Angola,-11.202700,17.873900
...,...,...,...
3999,Vietnam,14.058324,108.277199
4000,West Bank and Gaza,31.952200,35.233200
4001,Yemen,15.552727,48.516388
4002,Zambia,-13.133897,27.849332


## 透過條件判斷式以 `[布林陣列]` 篩選觀測值

In [23]:
is_taiwan = daily_report['Country_Region'] == 'Taiwan*'
print(is_taiwan)
daily_report[is_taiwan]

0       False
1       False
2       False
3       False
4       False
        ...  
3999    False
4000    False
4001    False
4002    False
4003    False
Name: Country_Region, Length: 4004, dtype: bool


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
663,NaN,NaN,NaN,Taiwan*,2021-09-24 04:21:44,23.7,121.0,16168,841,NaN,NaN,Taiwan*,67.884926,5.201633


## 同時選擇欄位以及篩選觀測值

In [24]:
cols_to_select = ['Country_Region', 'Confirmed']
rows_to_filter = daily_report['Country_Region'] == 'Taiwan*'
daily_report[rows_to_filter][cols_to_select]

,Country_Region,Confirmed
663,Taiwan*,16168


## 運用 `DataFrame` 的兩種索引語法取得資料

- `loc[index_label, column_label]`
- `iloc[index_position, column_position]`

## `loc[]` 以資料的列、欄標籤為準 

In [25]:
tokyo_loc = (daily_report[daily_report['Province_State'] == 'Tokyo']).index[0]
jp = daily_report[daily_report['Country_Region'] == 'Japan']
print(tokyo_loc)
jp.loc[tokyo_loc, ['Province_State', 'Country_Region', 'Confirmed']]

354


Province_State     Tokyo
Country_Region     Japan
Confirmed         373681
Name: 354, dtype: object

## `iloc[]` 以資料框的列、欄位置為準 

In [26]:
tokyo_iloc = [i for i, v in enumerate(jp.index) if v == tokyo_loc][0]
print(tokyo_iloc)
jp.iloc[tokyo_iloc, [2, 3, 7]]

41


Province_State     Tokyo
Country_Region     Japan
Confirmed         373681
Name: 354, dtype: object

## 使用（呼叫）資料框的兩種方法排序

- `sort_values`
- `sort_index`

## `sort_values` 方法可依指定欄位排序資料框

In [27]:
daily_report.sort_values('Confirmed')

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
1850,80022.0,Out of LA,Louisiana,US,2021-09-24 04:21:44,NaN,NaN,0,0,NaN,NaN,"Out of LA, Louisiana, US",NaN,NaN
3947,90056.0,Unassigned,Wyoming,US,2021-09-24 04:21:44,NaN,NaN,0,0,NaN,NaN,"Unassigned, Wyoming, US",NaN,NaN
2914,80040.0,Out of OK,Oklahoma,US,2021-09-24 04:21:44,NaN,NaN,0,0,NaN,NaN,"Out of OK, Oklahoma, US",NaN,NaN
745,2060.0,Bristol Bay,Alaska,US,2021-09-24 04:21:44,58.745140,-156.701064,0,0,NaN,NaN,"Bristol Bay, Alaska, US",0.000000,NaN
78,NaN,NaN,Diamond Princess,Canada,2020-12-21 13:27:30,NaN,NaN,0,1,NaN,NaN,"Diamond Princess, Canada",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,NaN,NaN,NaN,Iran,2021-09-24 04:21:44,32.427908,53.688046,5493591,118508,NaN,NaN,Iran,6540.537990,2.157205
3984,NaN,NaN,England,United Kingdom,2021-09-24 04:21:44,52.355500,-1.174300,6453164,118977,NaN,NaN,"England, United Kingdom",11528.200767,1.843700
269,NaN,NaN,Maharashtra,India,2021-09-24 04:21:44,19.449759,76.108221,6534557,138725,NaN,NaN,"Maharashtra, India",5306.425946,2.122944
216,NaN,NaN,NaN,France,2021-09-24 04:21:44,46.227600,2.213700,6810965,114209,NaN,NaN,France,10434.500598,1.676840


## `sort_index` 可依資料框的 `Index` 排序 

In [28]:
daily_report.sort_index(ascending=False)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
4003,NaN,NaN,NaN,Zimbabwe,2021-09-24 04:21:44,-19.015438,29.154857,129134,4600,NaN,NaN,Zimbabwe,868.832902,3.562191
4002,NaN,NaN,NaN,Zambia,2021-09-24 04:21:44,-13.133897,27.849332,208715,3641,NaN,NaN,Zambia,1135.310594,1.744484
4001,NaN,NaN,NaN,Yemen,2021-09-24 04:21:44,15.552727,48.516388,8830,1664,NaN,NaN,Yemen,29.605074,18.844847
4000,NaN,NaN,NaN,West Bank and Gaza,2021-09-24 04:21:44,31.952200,35.233200,392452,3980,NaN,NaN,West Bank and Gaza,7693.001316,1.014137
3999,NaN,NaN,NaN,Vietnam,2021-09-24 04:21:44,14.058324,108.277199,728435,18017,NaN,NaN,Vietnam,748.351761,2.473385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,NaN,NaN,NaN,Angola,2021-09-24 04:21:44,-11.202700,17.873900,53840,1460,NaN,NaN,Angola,163.815374,2.711738
3,NaN,NaN,NaN,Andorra,2021-09-24 04:21:44,42.506300,1.521800,15156,130,NaN,NaN,Andorra,19615.608620,0.857746
2,NaN,NaN,NaN,Algeria,2021-09-24 04:21:44,28.033900,1.659600,202283,5748,NaN,NaN,Algeria,461.295755,2.841564
1,NaN,NaN,NaN,Albania,2021-09-24 04:21:44,41.153300,20.168300,165864,2609,NaN,NaN,Albania,5763.569393,1.572975


## 從資料框變數衍生計算

- 簡單運算。
- 使用 Pandas 的 `cut` 函數。
- 使用 `Series` 的 `map` 方法。

## 簡單運算

In [29]:
active = daily_report['Confirmed'] - daily_report['Deaths'] - daily_report['Recovered']
print(active)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3999   NaN
4000   NaN
4001   NaN
4002   NaN
4003   NaN
Length: 4004, dtype: float64


## 使用 Pandas 的 `cut` 函數

In [30]:
cut_bins = [0, 1000, 10000, 100000, np.Inf]
cut_labels = ['Less than 1000', 'Between 1000 and 10000', 'Between 10000 and 100000', 'Above 100000']
confirmed_categorical = pd.cut(daily_report['Confirmed'], bins=cut_bins, labels=cut_labels, right=False)
print(confirmed_categorical)

0                   Above 100000
1                   Above 100000
2                   Above 100000
3       Between 10000 and 100000
4       Between 10000 and 100000
                  ...           
3999                Above 100000
4000                Above 100000
4001      Between 1000 and 10000
4002                Above 100000
4003                Above 100000
Name: Confirmed, Length: 4004, dtype: category
Categories (4, object): ['Less than 1000' < 'Between 1000 and 10000' < 'Between 10000 and 100000' < 'Above 100000']


## 使用 `Series` 的 `map` 方法

- 傳入一個 `dict`。
- 傳入一個函數（或 Lambda 函數）。

In [31]:
# Passing a dict
country_name = {
    'Taiwan*': 'Taiwan'
}
daily_report_tw = daily_report[is_taiwan]
daily_report_tw['Country_Region'].map(country_name)

663    Taiwan
Name: Country_Region, dtype: object

In [32]:
# Passing a function
def is_us(x):
    if x == 'US':
        return 'US'
    else:
        return 'Not US'
daily_report['Country_Region'].map(is_us)

0       Not US
1       Not US
2       Not US
3       Not US
4       Not US
         ...  
3999    Not US
4000    Not US
4001    Not US
4002    Not US
4003    Not US
Name: Country_Region, Length: 4004, dtype: object

In [33]:
# Passing a lambda expression)
daily_report['Country_Region'].map(lambda x: 'US' if x == 'US' else 'Not US')

0       Not US
1       Not US
2       Not US
3       Not US
4       Not US
         ...  
3999    Not US
4000    Not US
4001    Not US
4002    Not US
4003    Not US
Name: Country_Region, Length: 4004, dtype: object

## 如何新增資料框的變數

使用 `DataFrame` 的 `insert` 方法。

In [34]:
daily_report = get_latest_daily_report()
print(daily_report.columns)
new_column = daily_report['Confirmed'] - daily_report['Deaths'] - daily_report['Recovered']
daily_report.insert(0, "active", new_column)
print(daily_report.columns)

尚未有 09-24-2021 的每日報告。
載入了 09-23-2021 的每日報告。
Index(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incident_Rate', 'Case_Fatality_Ratio'],
      dtype='object')
Index(['active', 'FIPS', 'Admin2', 'Province_State', 'Country_Region',
       'Last_Update', 'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered',
       'Active', 'Combined_Key', 'Incident_Rate', 'Case_Fatality_Ratio'],
      dtype='object')


## 使用（呼叫）`Series` 的聚合方法摘要

In [35]:
daily_report['Confirmed'].sum()

230601553

## 使用（呼叫）`Series` 的 `groupby` 方法分組聚合

In [36]:
daily_report.groupby('Country_Region')['Confirmed'].sum()

Country_Region
Afghanistan           154800
Albania               165864
Algeria               202283
Andorra                15156
Angola                 53840
                       ...  
Vietnam               728435
West Bank and Gaza    392452
Yemen                   8830
Zambia                208715
Zimbabwe              129134
Name: Confirmed, Length: 195, dtype: int64

## 進階的資料框操作

## 進階的資料框操作包含

- 處理遺漏值（未定義值）。
- 處理文字變數。
- 轉置。
- 合併資料框。

## 處理遺漏值（未定義值）

- 使用（呼叫）`Series` 的 `isnull` 或者 `notnull` 方法來檢查遺漏值是否存在。
- 使用（呼叫）`DataFrame` 的 `dropna` 方法刪除有遺漏值的觀測值或欄位。
- 使用（呼叫）`Series` 的 `fillna` 方法填補遺漏值為指定資料。

In [37]:
print(daily_report['Province_State'].size)
print(daily_report['Province_State'].isnull().sum())
print(daily_report['Province_State'].notnull().sum())

4004
173
3831


In [38]:
print(daily_report.dropna().shape)
print(daily_report['FIPS'].fillna(0))

(0, 15)
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3999    0.0
4000    0.0
4001    0.0
4002    0.0
4003    0.0
Name: FIPS, Length: 4004, dtype: float64


## `Series` 摘要文字變數的方法

- `unique`
- `nunique`
- `value_counts`

In [39]:
print(daily_report['Country_Region'].nunique())
print(daily_report['Country_Region'].unique())

195
['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Chile' 'China' 'Colombia' 'Comoros' 'Congo (Brazzaville)'
 'Congo (Kinshasa)' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus'
 'Czechia' 'Denmark' 'Diamond Princess' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Holy See'
 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan'

In [40]:
daily_report['Country_Region'].value_counts().sort_values(ascending=False)

US          3278
Russia        83
Japan         49
India         37
China         34
            ... 
Ethiopia       1
Fiji           1
Zambia         1
Botswana       1
Ireland        1
Name: Country_Region, Length: 195, dtype: int64

## 使用（呼叫）`Series` 的 `str.split` 方法切割文字

In [41]:
request_url = "https://www.imdb.com/chart/top"
html_tables = pd.read_html(request_url)
html_table = html_tables[0]
html_table.head()

,Unnamed: 0,Rank & Title,IMDb Rating,Your Rating,Unnamed: 4
0,NaN,1. 刺激1995 (1994),9.2,12345678910 NOT YET RELEASED Seen,NaN
1,NaN,2. 教父 (1972),9.1,12345678910 NOT YET RELEASED Seen,NaN
2,NaN,3. 教父第二集 (1974),9.0,12345678910 NOT YET RELEASED Seen,NaN
3,NaN,4. 黑暗騎士 (2008),9.0,12345678910 NOT YET RELEASED Seen,NaN
4,NaN,5. 十二怒漢 (1957),8.9,12345678910 NOT YET RELEASED Seen,NaN


In [42]:
html_table['Rank & Title'].str.split() # split as a Series

0          [1., 刺激1995, (1994)]
1              [2., 教父, (1972)]
2           [3., 教父第二集, (1974)]
3            [4., 黑暗騎士, (2008)]
4            [5., 十二怒漢, (1957)]
                 ...           
245        [246., 公主新娘, (1987)]
246    [247., Ratsasan, (2018)]
247       [248., 男人的爭鬥, (1955)]
248      [249., 卡比莉亞之夜, (1957)]
249          [250., 日出, (1927)]
Name: Rank & Title, Length: 250, dtype: object

In [43]:
html_table['Rank & Title'].str.split(expand=True) # split as a DataFrame

,0,1,2,3,4
0,1.,刺激1995,(1994),None,None
1,2.,教父,(1972),None,None
2,3.,教父第二集,(1974),None,None
3,4.,黑暗騎士,(2008),None,None
4,5.,十二怒漢,(1957),None,None
...,...,...,...,...,...
245,246.,公主新娘,(1987),None,None
246,247.,Ratsasan,(2018),None,None
247,248.,男人的爭鬥,(1955),None,None
248,249.,卡比莉亞之夜,(1957),None,None


## 使用（呼叫）`Series` 的 `str.replace` 方法取代文字中的指定特徵

In [44]:
split_dataframe = html_table['Rank & Title'].str.split(expand=True)
ranks = split_dataframe[0].str.replace(".", '', regex=False)
ranks

0        1
1        2
2        3
3        4
4        5
      ... 
245    246
246    247
247    248
248    249
249    250
Name: 0, Length: 250, dtype: object

In [45]:
split_dataframe = html_table['Rank & Title'].str.split(expand=True)
years = split_dataframe[2].str.replace("\(|\)", '', regex=True)
years

0      1994
1      1972
2      1974
3      2008
4      1957
       ... 
245    1987
246    2018
247    1955
248    1957
249    1927
Name: 2, Length: 250, dtype: object

## 使用（呼叫）`Series` 的 `str.contains` 方法判斷文字是否包含指定特徵

In [46]:
print(daily_report['Country_Region'].str.contains('land').sum())
daily_report[daily_report['Country_Region'].str.contains('land')].head()

27


,active,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
204,NaN,NaN,NaN,NaN,Finland,2021-09-24 04:21:44,61.92411,25.748151,138068,1062,NaN,NaN,Finland,2491.879211,0.769186
248,NaN,NaN,NaN,NaN,Iceland,2021-09-24 04:21:44,64.96310,-19.020800,11594,33,NaN,NaN,Iceland,3397.509158,0.284630
289,NaN,NaN,NaN,NaN,Ireland,2021-09-24 04:21:44,53.14240,-7.692100,380720,5209,NaN,NaN,Ireland,7710.322581,1.368197
402,NaN,NaN,NaN,NaN,Marshall Islands,2021-09-24 04:21:44,7.13150,171.184500,4,0,NaN,NaN,Marshall Islands,6.847791,0.000000
447,NaN,NaN,NaN,Aruba,Netherlands,2021-09-24 04:21:44,12.52110,-69.968300,15351,160,NaN,NaN,"Aruba, Netherlands",14378.172827,1.042277


## 轉置資料框的函數與方法

- Pandas 的 `melt` 函數可以將資料框由寬格式（Wide format）轉換為長格式（Long format）
- `DataFrame` 的 `pivot` 方法可以將資料框由長格式（Long format）轉換為寬格式（Wide format）。

## 什麼是寬格式、長格式？

- 寬格式使用一個欄位，欄位名稱記錄變數類別、觀測值記錄其數值。 
- 長格式使用兩個欄位，一個記錄變數類別、一個記錄數值。

來源：<https://en.wikipedia.org/wiki/Wide_and_narrow_data>

## 為何需要轉置資料框？

資料框的欄位名稱含有使用者需要的資料值或者儲存格式與應用情境不符。

In [47]:
ts_confirmed_global_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master\
/csse_covid_19_data/csse_covid_19_time_series/\
time_series_covid19_confirmed_global.csv"
ts_confirmed_global = pd.read_csv(ts_confirmed_global_url)
ts_confirmed_global

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/14/21,9/15/21,9/16/21,9/17/21,9/18/21,9/19/21,9/20/21,9/21/21,9/22/21,9/23/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,154180,154283,154361,154487,154487,154487,154585,154712,154757,154800
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,158431,159423,160365,161324,162173,162953,163404,164276,165096,165864
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,200528,200770,200989,201224,201425,201600,201766,201948,202122,202283
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,15099,15108,15113,15124,15124,15124,15140,15140,15153,15156
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,51047,51407,51827,52208,52307,52307,52644,52968,53387,53840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,635055,645640,656129,667650,677023,687063,695744,707436,718963,728435
275,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,374768,377134,379635,381854,382584,384390,386493,388470,390369,392452
276,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,8502,8527,8557,8593,8630,8667,8718,8752,8789,8830
277,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,208049,208161,208267,208353,208422,208469,208502,208599,208676,208715


## 使用（呼叫）Pandas 的 `melt` 函數寬轉長

- `id_vars`: `['Province/State', 'Country/Region', 'Lat', 'Long']`
- `value_vars`: 從 `1/22/20` 開始到最後一個欄位。
- `var_name`: 自訂為 `Date`。
- `value_name`: 自訂為 `Confirmed`。

In [48]:
idVars = ['Province/State', 'Country/Region', 'Lat', 'Long']
ts_confirmed_global_long = pd.melt(ts_confirmed_global,
                                  id_vars=idVars,
                                  var_name='Date',
                                  value_name='Confirmed')
ts_confirmed_global_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
170464,NaN,Vietnam,14.058324,108.277199,9/23/21,728435
170465,NaN,West Bank and Gaza,31.952200,35.233200,9/23/21,392452
170466,NaN,Yemen,15.552727,48.516388,9/23/21,8830
170467,NaN,Zambia,-13.133897,27.849332,9/23/21,208715


In [49]:
ts_groupby_confirmed = ts_confirmed_global_long.groupby(['Country/Region', 'Date'])['Confirmed'].sum()
ts_groupby_confirmed = pd.DataFrame(ts_groupby_confirmed).reset_index()
ts_groupby_confirmed

,Country/Region,Date,Confirmed
0,Afghanistan,1/1/21,52513
1,Afghanistan,1/10/21,53489
2,Afghanistan,1/11/21,53538
3,Afghanistan,1/12/21,53584
4,Afghanistan,1/13/21,53690
...,...,...,...
119140,Zimbabwe,9/7/21,125796
119141,Zimbabwe,9/8/20,7388
119142,Zimbabwe,9/8/21,125931
119143,Zimbabwe,9/9/20,7429


## 使用（呼叫）`DataFrame` 的 `pivot` 方法長轉寬

In [50]:
ts_groupby_confirmed.pivot(index='Country/Region', columns='Date', values='Confirmed')

Date,1/1/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,...,9/5/20,9/5/21,9/6/20,9/6/21,9/7/20,9/7/21,9/8/20,9/8/21,9/9/20,9/9/21
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,52513,53489,53538,53584,53690,53775,53831,53938,53984,54062,...,38390,153423,38484,153534,38580,153626,38606,153736,38630,153840
Albania,58316,63595,63971,64627,65334,65994,66635,67216,67690,67982,...,10102,150997,10255,151499,10406,152239,10553,153318,10704,154316
Algeria,99897,102144,102369,102641,102860,103127,103381,103611,103833,104092,...,46071,198004,46364,198313,46653,198645,46938,198962,47216,199275
Andorra,8117,8586,8586,8682,8818,8868,8946,9038,9083,9083,...,1215,15055,1215,15069,1261,15070,1261,15070,1301,15078
Angola,17568,18193,18254,18343,18425,18613,18679,18765,18875,18926,...,2935,48656,2965,48790,2981,49114,3033,49349,3092,49628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,1474,1514,1515,1520,1521,1531,1536,1537,1537,1539,...,1049,524307,1049,536788,1049,550996,1054,563676,1059,576096
West Bank and Gaza,139223,147400,148171,148968,149769,150505,151142,151569,152031,152555,...,25575,353732,26127,356474,26779,359236,27363,361881,27919,364364
Yemen,2101,2104,2105,2107,2109,2110,2111,2112,2112,2113,...,1983,8108,1987,8142,1989,8181,1994,8230,1999,8267


## 合併資料框的函數與方法

- 使用（呼叫）Pandas 的 `merge` 函數利用欄位名稱合併。
- 使用（呼叫）`DataFrame` 的 `join` 方法利用資料框的列索引合併。

In [51]:
def get_top_rated_movies():
    request_url = "https://www.imdb.com/chart/top"
    html_tables = pd.read_html(request_url)
    html_table = html_tables[0]
    split_dataframe = html_table['Rank & Title'].str.split(expand=True)
    ranks = split_dataframe[0].str.replace(".", '', regex=False).values
    years = split_dataframe[2].str.replace("\(|\)", '', regex=True).values
    titles = split_dataframe[1].values
    ratings = html_table['IMDb Rating'].values
    out = pd.DataFrame()
    out['rank'] = ranks
    out['title'] = titles
    out['year'] = years
    out['rating'] = ratings
    return out

top_rated_movies = get_top_rated_movies()

In [52]:
left_df = top_rated_movies.loc[:9, ['title', 'year']]
right_df = top_rated_movies[top_rated_movies['title'].str.contains('魔戒')][['title', 'rating']].reset_index(drop=True)
left_df

,title,year
0,刺激1995,1994
1,教父,1972
2,教父第二集,1974
3,黑暗騎士,2008
4,十二怒漢,1957
5,辛德勒的名單,1993
6,魔戒三部曲：王者再臨,2003
7,黑色追緝令,1994
8,黃昏三鏢客,1966
9,魔戒首部曲：魔戒現身,2001


In [53]:
right_df

,title,rating
0,魔戒三部曲：王者再臨,8.9
1,魔戒首部曲：魔戒現身,8.8
2,魔戒二部曲：雙城奇謀,8.7


## 使用（呼叫）Pandas 的 `merge` 函數利用欄位名稱合併

In [54]:
# default: inner join
pd.merge(left_df, right_df)

,title,year,rating
0,魔戒三部曲：王者再臨,2003,8.9
1,魔戒首部曲：魔戒現身,2001,8.8


In [55]:
# left join
pd.merge(left_df, right_df, how='left')

,title,year,rating
0,刺激1995,1994,NaN
1,教父,1972,NaN
2,教父第二集,1974,NaN
3,黑暗騎士,2008,NaN
4,十二怒漢,1957,NaN
5,辛德勒的名單,1993,NaN
6,魔戒三部曲：王者再臨,2003,8.9
7,黑色追緝令,1994,NaN
8,黃昏三鏢客,1966,NaN
9,魔戒首部曲：魔戒現身,2001,8.8


In [56]:
# right join
pd.merge(left_df, right_df, how='right')

,title,year,rating
0,魔戒三部曲：王者再臨,2003,8.9
1,魔戒首部曲：魔戒現身,2001,8.8
2,魔戒二部曲：雙城奇謀,NaN,8.7


In [57]:
# outer join
pd.merge(left_df, right_df, how='outer')

,title,year,rating
0,刺激1995,1994,NaN
1,教父,1972,NaN
2,教父第二集,1974,NaN
3,黑暗騎士,2008,NaN
4,十二怒漢,1957,NaN
5,辛德勒的名單,1993,NaN
6,魔戒三部曲：王者再臨,2003,8.9
7,黑色追緝令,1994,NaN
8,黃昏三鏢客,1966,NaN
9,魔戒首部曲：魔戒現身,2001,8.8


## 使用（呼叫）DataFrame 的 `join` 方法利用資料框的列索引合併

In [58]:
# join dataframes on index
left_df.join(right_df, lsuffix='_x', rsuffix='_y')

,title_x,year,title_y,rating
0,刺激1995,1994,魔戒三部曲：王者再臨,8.9
1,教父,1972,魔戒首部曲：魔戒現身,8.8
2,教父第二集,1974,魔戒二部曲：雙城奇謀,8.7
3,黑暗騎士,2008,NaN,NaN
4,十二怒漢,1957,NaN,NaN
5,辛德勒的名單,1993,NaN,NaN
6,魔戒三部曲：王者再臨,2003,NaN,NaN
7,黑色追緝令,1994,NaN,NaN
8,黃昏三鏢客,1966,NaN,NaN
9,魔戒首部曲：魔戒現身,2001,NaN,NaN


In [59]:
left_df = left_df.set_index('title')
right_df = right_df.set_index('title')
# default: left join
left_df.join(right_df)

,year,rating
title,,
刺激1995,1994,NaN
教父,1972,NaN
教父第二集,1974,NaN
黑暗騎士,2008,NaN
十二怒漢,1957,NaN
辛德勒的名單,1993,NaN
魔戒三部曲：王者再臨,2003,8.9
黑色追緝令,1994,NaN
黃昏三鏢客,1966,NaN


In [60]:
# inner join
left_df.join(right_df, how='inner')

,year,rating
title,,
魔戒三部曲：王者再臨,2003,8.9
魔戒首部曲：魔戒現身,2001,8.8


In [61]:
# right join
left_df.join(right_df, how='right')

,year,rating
title,,
魔戒三部曲：王者再臨,2003,8.9
魔戒首部曲：魔戒現身,2001,8.8
魔戒二部曲：雙城奇謀,NaN,8.7


In [62]:
# outer join
left_df.join(right_df, how='outer')

,year,rating
title,,
刺激1995,1994,NaN
十二怒漢,1957,NaN
教父,1972,NaN
教父第二集,1974,NaN
辛德勒的名單,1993,NaN
魔戒三部曲：王者再臨,2003,8.9
魔戒二部曲：雙城奇謀,NaN,8.7
魔戒首部曲：魔戒現身,2001,8.8
黃昏三鏢客,1966,NaN
